### Creating a list of window coordiantes

Extent:  
99000,6702000 : 402000,6882000

In [1]:
im_path_spot_2 = '/home/simon/CDE_UBS/thesis/data_collection/spot6/spot6_mosaic.tif'

In [16]:
import rasterio
import pandas as pd
import numpy as np

In [3]:
def get_spatial_extent(filepath):
    """
    Takes filepath, returns bounding box
    """
    
    import rasterio
    
    with rasterio.open(filepath) as src:
        bbox = src.bounds
    return(bbox)

In [155]:
def create_window_coordinates(filepath,window_size=500,clip=True):
    """
    Takes filepath, creates grid of coordinate points in wanted window size.
    (sampling of points bc mask reads whole into RAM)
    """
    
    # get bbox
    bbox = get_spatial_extent(filepath)
    left = int(bbox[0])
    bottom = int(bbox[1])
    right = int(bbox[2])
    top = int(bbox[3])
    
    # iterate in 500 steps over image bounds, create grid
    coor = []
    for i in range(left,right,window_size):
        x = i
        for j in range(bottom,top,window_size):
            y = j
            coor.append((x,y))
    
    
    """
    PERFORM CLIP
    """
    if clip:
        import geopandas
        # load into gdf
        print("performing clip!\nloading into gdf...")
        df = pd.DataFrame(coor,columns=["x","y"])
        gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.x, df.y))

        print("start sampling...")
        with rasterio.open(filepath) as src:
            gdf['value'] = [sum(x) for x in src.sample(coor)]
            
        # drop invalid points and useless columns
        gdf = gdf.drop(gdf[gdf.value <= 0].index)
        gdf = gdf[["x","y"]]
        
        # set coor to clipped points, convert back to tuple
        records = gdf.to_records(index=False)
        coor = list(records)
    
    return(coor)
    

In [ ]:
a = create_window_coordinates(im_path_spot_2,clip=True)

In [ ]:
df = pd.DataFrame(create_window_coordinates(im_path_spot_2),columns=["x","y"])

performing clip!
loading into gdf...
start sampling...


In [ ]:
df.to_csv("test_coors.csv")

In [13]:
with rasterio.open(im_path_spot_2) as src:
    bbox = src.bounds
    print(bbox)

BoundingBox(left=99000.0, bottom=6702000.0, right=402000.0, top=6882000.0)
